## Initialization

In [1]:
# !pip install intel_extension_for_pytorch

In [2]:
# import sys
# import site
# from pathlib import Path

# # !echo "Installation in progress, please wait..."
# # !{sys.executable} -m pip cache purge > /dev/null
# # !{sys.executable} -m pip install peft==0.5.0 --no-deps > /dev/null
# # !{sys.executable} -m pip install accelerate==0.23.0 --no-deps --no-warn-script-location > /dev/null
# # !{sys.executable} -m pip install  transformers==4.34.0 --no-warn-script-location > /dev/null 
# # !{sys.executable} -m pip install datasets==2.15 --no-warn-script-location > /dev/null 2>&1 
# # !{sys.executable} -m pip install fsspec==2023.9.2 > /dev/null  2>&1

# # removed
# #!{sys.executable} -m pip install --pre bigdl-llm[xpu]==2.4.0b20231116 --no-warn-script-location -f https://developer.intel.com/ipex-whl-stable-xpu > /dev/null


# # added
# # !{sys.executable} -m pip install bigdl_core_xe
# !pip install BigDL
# !pip install BigDL-llm
# # !pip install bigdl_core_xe

# !echo "Installation completed."

# def get_python_version():
#     return "python" + ".".join(map(str, sys.version_info[:2]))

# def set_local_bin_path():
#     local_bin = str(Path.home() / ".local" / "bin") 
#     local_site_packages = str(
#         Path.home() / ".local" / "lib" / get_python_version() / "site-packages"
#     )
#     sys.path.append(local_bin)
#     sys.path.insert(0, site.getusersitepackages())
#     sys.path.insert(0, sys.path.pop(sys.path.index(local_site_packages)))

# set_local_bin_path()

In [3]:
import logging
import os
import sys
from math import ceil
from typing import Optional, Tuple
import warnings

warnings.filterwarnings(
    "ignore", category=UserWarning, module="intel_extension_for_pytorch"
)
warnings.filterwarnings(
    "ignore", category=UserWarning, module="torchvision.io.image", lineno=13
)
warnings.filterwarnings("ignore", message="You are using the default legacy behaviour")
warnings.filterwarnings("ignore", category=UserWarning, message=".*Parameter.*")
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="This implementation of AdamW is deprecated",
)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["NUMEXPR_MAX_THREADS"] = "28"
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("bigdl").setLevel(logging.ERROR)


import torch
import intel_extension_for_pytorch as ipex
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from bigdl.llm.transformers import AutoModelForCausalLM
from bigdl.llm.transformers.qlora import (
    get_peft_model,
    prepare_model_for_kbit_training as prepare_model,
)
from peft import LoraConfig
from bigdl.llm.transformers.qlora import PeftModel
import transformers
from transformers import (
    DataCollatorForSeq2Seq,
    LlamaTokenizer,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    
)

transformers.logging.set_verbosity_error()

## Helper Functions

In [4]:
def generate_prompt_sql(input_question, context, output=""):
    """
    Generates a prompt for fine-tuning the LLM model for text-to-SQL tasks.

    Parameters:
        input_question (str): The input text or question to be converted to SQL.
        context (str): The schema or context in which the SQL query operates.
        output (str, optional): The expected SQL query as the output.

    Returns:
        str: A formatted string serving as the prompt for the fine-tuning task.
    """
    return f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. 

You must output the SQL query that answers the question.

### Input:
{input_question}

### Context:
{context}

### Response:
{output}"""

In [5]:
def setup_model_and_tokenizer(base_model_id: str, model_cache_path):
    """Downloads / Loads the pre-trained model in NF4 datatype and tokenizer based on the given base model ID for training."""
    local_model_id = base_model_id.replace("/", "--")
    local_model_path = os.path.join(model_cache_path, local_model_id)
    print(f"local model path is: {local_model_path}")

    try:
        model = AutoModelForCausalLM.from_pretrained(
            local_model_path,
            load_in_low_bit="nf4",
            optimize_model=False,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )
    except OSError as e:
        print(e)
        sys.exit()
        logging.info(
            f"Model not found locally. Downloading {base_model_id} to cache..."
        )
        model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            load_in_low_bit="nf4",
            optimize_model=False,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )

    try:
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(local_model_path)
        else:
            tokenizer = AutoTokenizer.from_pretrained(local_model_path)
    except OSError:
        logging.info(
            f"Tokenizer not found locally. Downloading tokenizer for {base_model_id} to cache..."
        )
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(base_model_id)
        else:
            tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    tokenizer.pad_token_id = 0
    tokenizer.padding_side = "left"
    return model, tokenizer

---

**FineTuner**

The `FineTuner` class encapsulates the entire process of fine-tuning llms for tasks such as text-to-SQL conversion.


**Tokenization Strategy**

The tokenization process is tailored to the type of model being fine-tuned. For instance, if we are working with a Llama model, we utilize a `LlamaTokenizer` to ensure compatibility with the model's expected input format. For other models, a generic `AutoTokenizer` is used. We configure the tokenizer to pad from the left side (`padding_side="left"`) and set the pad token ID to 0.

**Data Tokenization and Preparation**

The `tokenize_data` method is where the fine-tuner ingests raw text data and converts it into a format suitable for training the model. This method handles the addition of end-of-sequence tokens, truncation to a specified `cutoff_len`, and conditioning on the input for training.

**Dataset Handling**

`prepare_data` manages the splitting of data into training and validation sets, applying the `tokenize_data` transformation to each entry. This ensures that our datasets are ready for input into the model, with all necessary tokenization applied.

**Training Process**

Finally, the `train_model` method orchestrates the training process, setting up the `Trainer` with the correct datasets, training arguments, and data collator. The fine-tuning process is encapsulated within the `finetune` method, which strings together all the previous steps into a coherent pipeline, from model setup to training execution.

**Using QLoRA for Efficient Fine-Tuning**
1. Load a pretrained model (e.g., LLaMA) in low precision with `load_in_low_bit="nf4"` for 4-bit quantized weights.
2. Prepare the quantized model with `prepare_model(model)`, handling weight quantization.
3. Add LoRA adapters via `get_peft_model(model, config)` for setting adapter parameters.
4. Fine-tune with `Trainer`, focusing gradients on adapters while keeping base model weights fixed.

**Code Implementation**
- Model loading with BigDL's `AutoModelForCausalLM`, initializing in 4-bit using `load_in_low_bit="nf4"`.
- `prepare_model()` quantizes the model weights.
- `get_peft_model()` adds LoRA adapters.
- Trainer handles fine-tuning, optimizing only adapter weights.


So in summary, we leverage QLoRA in BigDL to load the base LLM in low precision, inject adapters with `peft`, and efficiently finetune by optimizing just the adapters end-to-end while keeping the base model fixed. This unlocks huge memory savings, allowing us to adapt giant models.

## Fine-Tuning Pipeline

In [6]:
class FineTuner:
    """A class to handle the fine-tuning of LLM models."""

    def __init__(self, base_model_id: str, model_path: str, device: torch.device, model_cache_path):
        """
        Initialize the FineTuner with base model, model path, and device.

        Parameters:
            base_model_id (str): Id of pre-trained model to use for fine-tuning.
            model_path (str): Path to save the fine-tuned model.
            device (torch.device): Device to run the model on.
        """
        self.base_model_id = base_model_id
        self.model_path = model_path
        self.device = device
        self.model, self.tokenizer = setup_model_and_tokenizer(base_model_id, model_cache_path)


    def tokenize_data(
        self, data_points, add_eos_token=True, train_on_inputs=False, cutoff_len=512
    ) -> dict:
        """
        Tokenizes dataset of SQL related data points consisting of questions, context, and answers.

        Parameters:
            data_points (dict): A batch from the dataset containing 'question', 'context', and 'answer'.
            add_eos_token (bool): Whether to add an EOS token at the end of each tokenized sequence.
            cutoff_len (int): The maximum length for each tokenized sequence.

        Returns:
            dict: A dictionary containing tokenized 'input_ids', 'attention_mask', and 'labels'.
        """
        try:
            question = data_points["question"]
            context = data_points["context"]
            answer = data_points["answer"]
            if train_on_inputs:
                user_prompt = generate_prompt_sql(question, context)
                tokenized_user_prompt = self.tokenizer(
                    user_prompt,
                    truncation=True,
                    max_length=cutoff_len,
                    padding=False,
                    return_tensors=None,
                )
                user_prompt_len = len(tokenized_user_prompt["input_ids"])
                if add_eos_token:
                    user_prompt_len -= 1

            combined_text = generate_prompt_sql(question, context, answer)
            tokenized = self.tokenizer(
                combined_text,
                truncation=True,
                max_length=cutoff_len,
                padding=False,
                return_tensors=None,
            )
            if (
                tokenized["input_ids"][-1] != self.tokenizer.eos_token_id
                and add_eos_token
                and len(tokenized["input_ids"]) < cutoff_len
            ):
                tokenized["input_ids"].append(self.tokenizer.eos_token_id)
                tokenized["attention_mask"].append(1)
            tokenized["labels"] = tokenized["input_ids"].copy()
            if train_on_inputs:
                tokenized["labels"] = [-100] * user_prompt_len + tokenized["labels"][
                    user_prompt_len:
                ]

            return tokenized
        except Exception as e:
            logging.error(
                f"Error in batch tokenization: {e}, Line: {e.__traceback__.tb_lineno}"
            )
            raise e

    def prepare_data(self, data, val_set_size=100) -> Dataset:
        """Prepare training and validation datasets."""
        try:
            train_val_split = data["train"].train_test_split(
                test_size=val_set_size, shuffle=True, seed=42
            )
            train_data = train_val_split["train"].shuffle().map(self.tokenize_data)
            val_data = train_val_split["test"].shuffle().map(self.tokenize_data)
            return train_data, val_data
        except Exception as e:
            logging.error(
                f"Error in preparing data: {e}, Line: {e.__traceback__.tb_lineno}"
            )
            raise e

    def train_model(self, train_data, val_data, training_args, lora_config):
        """
        Fine-tune the model with the given training and validation data.

        Parameters:
            train_data (Dataset): Training data.
            val_data (Optional[Dataset]): Validation data.
            training_args (TrainingArguments): Training configuration.
        """
        try:
            self.model = self.model.to(self.device)
            self.model = prepare_model(self.model)
            self.model = get_peft_model(self.model, lora_config)
            trainer = Trainer(
                model=self.model,
                train_dataset=train_data,
                eval_dataset=val_data,
                args=training_args,
                data_collator=DataCollatorForSeq2Seq(
                    self.tokenizer,
                    pad_to_multiple_of=8,
                    return_tensors="pt",
                    padding=True,
                ),
            )
            self.model.config.use_cache = False
            trainer.train()
            self.model.save_pretrained(self.model_path)
        except Exception as e:
            logging.error(f"Error in model training: {e}")

    def finetune(self, data_path, training_args, lora_config):
        """
        Execute the fine-tuning pipeline.

        Parameters:
            data_path (str): Path to the data for fine-tuning.
            training_args (TrainingArguments): Training configuration.
        """
        try:
            data = load_dataset(data_path)
            train_data, val_data = self.prepare_data(data)
            self.train_model(train_data, val_data, training_args, lora_config)
        except KeyboardInterrupt:
            print("Interrupt received, saving model...")
            self.model.save_pretrained(f"{self.model_path}_interrupted")
            print(f"Model saved to {self.model_path}_interrupted")
        except Exception as e:
            logging.error(f"Error in fintuning: {e}")

In [7]:
ENABLE_WANDB = False

def lets_finetune(
    device,#=DEVICE,
    model,#=BASE_MODEL,
    MODEL_PATH,# = MODEL_PATH
    model_cache_path, # not here before
    data_path,
    lora_config,
    per_device_batch_size=4,
    warmup_steps=20,
    learning_rate=2e-5,
    max_steps=100,
    gradient_accum_steps=4,
    
    
):
    try:
        # Training parameters
        save_steps = 20
        eval_steps = 20
        max_grad_norm = 0.3
        save_total_limit = 3
        logging_steps = 20

        # Initialize the finetuner with the model and device information
        finetuner = FineTuner(
            base_model_id=model, model_path=MODEL_PATH, device=device, model_cache_path = model_cache_path
        )

        training_args = TrainingArguments(
            per_device_train_batch_size=per_device_batch_size,
            gradient_accumulation_steps=gradient_accum_steps,
            warmup_steps=warmup_steps,
            save_steps=save_steps,
            save_strategy="steps",
            eval_steps=eval_steps,
            evaluation_strategy="steps",
            max_steps=max_steps,
            learning_rate=learning_rate,
            #max_grad_norm=max_grad_norm,
            bf16=True,
            #lr_scheduler_type="cosine",
            load_best_model_at_end=True,
            ddp_find_unused_parameters=False,
            group_by_length=True,
            save_total_limit=save_total_limit,
            logging_steps=logging_steps,
            optim="adamw_hf",
            output_dir=f"./lora_adapters/{model}",
            logging_dir="./logs",
            report_to="wandb" if ENABLE_WANDB else [],
        )

        # Start fine-tuning
        finetuner.finetune(data_path, training_args, lora_config)
    except Exception as e:
        logging.error(f"Error occurred: {e}")

## Testing Pipeline

In [8]:
os.environ["WANDB_DISABLED"] = "true"
INFERENCE_DEVICE = torch.device("cpu")  # change this to `xpu` to use Intel GPU for inference  

def generate_prompt_sql(input_question, context, output=""):
    """
    Generates a prompt for fine-tuning the LLM model for text-to-SQL tasks.

    Parameters:
        input_question (str): The input text or question to be converted to SQL.
        context (str): The schema or context in which the SQL query operates.
        output (str, optional): The expected SQL query as the output.

    Returns:
        str: A formatted string serving as the prompt for the fine-tuning task.
    """
    return f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. 

You must output the SQL query that answers the question.

### Input:
{input_question}

### Context:
{context}

### Response:
{output}"""


def setup_model_and_tokenizer(base_model_id: str, model_cache_path):
    """Downloads / Load the pre-trained model in 4bit and tokenizer based on the given base model ID for inference."""
    local_model_id = base_model_id.replace("/", "--")
    local_model_path = os.path.join(model_cache_path, local_model_id)
    print(f"local model path is: {local_model_path}")

    try:
        model = AutoModelForCausalLM.from_pretrained(
            local_model_path,
            load_in_4bit=True,
            optimize_model=True,
            use_cache=True,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )
    except OSError:
        logging.info(
            f"Model not found locally. Downloading {base_model_id} to cache..."
        )
        model = AutoModelForCausalLM.from_pretrained(
            local_model_path,
            load_in_4bit=True,
            optimize_model=True,
            use_cache=True,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )

    try:
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(local_model_path)
        else:
            tokenizer = AutoTokenizer.from_pretrained(local_model_path)
    except OSError:
        logging.info(
            f"Tokenizer not found locally. Downloading tokenizer for {base_model_id} to cache..."
        )
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(base_model_id)
        else:
            tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    tokenizer.pad_token_id = 0
    tokenizer.padding_side = "left"
    return model, tokenizer

class TextToSQLGenerator:
    """Handles SQL query generation for a given text prompt."""

    def __init__(
        self, base_model_id, model_cache_path,#=BASE_MODEL, 
        use_adapter=False, lora_checkpoint=None, loaded_base_model=None
    ):
        """
        Initialize the InferenceModel class.
        Parameters:
            use_adapter (bool, optional): Whether to use LoRA model. Defaults to False.
        """
        try:
            if loaded_base_model:
                self.model = loaded_base_model.model
                self.tokenizer = loaded_base_model.tokenizer
            else:
                self.model, self.tokenizer = setup_model_and_tokenizer(base_model_id, model_cache_path=model_cache_path)
            if use_adapter:
                self.model = PeftModel.from_pretrained(self.model, lora_checkpoint)
        except Exception as e:
            logging.error(f"Exception occurred during model initialization: {e}")
            raise

        self.model.to(INFERENCE_DEVICE)
        self.max_length = 512


    def generate(self, prompt, **kwargs):
        """Generates an SQL query based on the given prompt.
        Parameters:
            prompt (str): The SQL prompt.
        Returns:
            str: The generated SQL query.
        """
        try:
            encoded_prompt = self.tokenizer(
                prompt,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt",
            ).input_ids.to(INFERENCE_DEVICE)
            with torch.no_grad():
                with torch.xpu.amp.autocast():
                    outputs = self.model.generate(
                        input_ids=encoded_prompt,
                        do_sample=True,
                        max_length=self.max_length,
                        temperature=0.3,
                        repetition_penalty=1.2,
                    )
            generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return generated
        except Exception as e:
            logging.error(f"Exception occurred during query generation: {e}")
            raise

In [9]:
import json

from IPython.display import display, HTML

INFERENCE_DEVICE = torch.device("cpu")  


# let's use some fake sample data
samples = """
[
  {
    "question": "What is the capacity of the stadium where the team 'Mountain Eagles' plays?",
    "context": "CREATE TABLE stadium_info (team_name VARCHAR, stadium_name VARCHAR, capacity INT)",
    "target": "SELECT capacity FROM stadium_info WHERE team_name = 'Mountain Eagles';"
  },
  {
    "question": "How many goals did player John Smith score last season?",
    "context": "CREATE TABLE player_stats (player_name VARCHAR, goals_scored INT, season VARCHAR)",
    "target": "SELECT SUM(goals_scored) FROM player_stats WHERE player_name = 'John Smith' AND season IN ('last season');"
  },
  {
    "question": "What are the operating hours for the Central Library on weekends?",
    "context": "CREATE TABLE library_hours (library_name VARCHAR, day_of_week VARCHAR, open_time TIME, close_time TIME)",
    "target": "SELECT * FROM library_hours WHERE library_name = 'Central' AND day_of_week IN ('Saturday', 'Sunday');"
  }
]
"""

def _extract_sections(output):
    input_section = output.split("### Input:")[1].split("### Context:")[0]
    context_section = output.split("### Context:")[1].split("### Response:")[0]
    response_section = output.split("### Response:")[1]
    return input_section, context_section, response_section

def run_inference(sample_data, model, finetuned=False):
    if INFERENCE_DEVICE.type.startswith("xpu"):
        torch.xpu.empty_cache()
    
    color = "#4CAF52" if finetuned else "#2196F4"
    model_type = "finetuned" if finetuned else "base"
    display(HTML(f"<div style='color:{color};'>Processing queries on {INFERENCE_DEVICE} please wait...</div>"))

    
    for index, row in enumerate(sample_data):
        try:
            prompt = generate_prompt_sql(row["question"], context=row["context"])
            output = model.generate(prompt)            
            input_section, context_section, response_section = _extract_sections(output)
            
            tabbed_output = f"""
            <details>
                <summary style='color: {color};'><b>{model_type} model - Sample {index+1}</b> (Click to expand)</summary>
                <div style='padding-left: 20px;'>
                    <p><b>Expected input 📝:</b><br>{input_section}</p>
                    <p><b>Expected context 📚:</b><br>{context_section}</p>
                    <p><b>Generated response 💡:</b><br>{response_section}</p>
                    <p><b>Target response 🎯:</b><br>{row["target"]}</p>
                    <p><b>Correct?:</b><br>{row["target"] in response_section}</p>
                </div>
            </details>
            <hr style='border-top: 1px solid #bbb;'>"""  # Subtle separator
            display(HTML(tabbed_output))
        except Exception as e:
            logging.error(f"Exception occurred during sample processing: {e}")

## Generate Model

In [10]:
def generate_model(model):
    print("Model:",model)
    model = model.replace("/", "--")
    MODEL_PATH = f"./final_model/{model}"
    BASE_MODEL = model
    #DEVICE = torch.device("xpu" if torch.xpu.is_available() else "cpu")
    device = torch.device("xpu")
    data_path = "b-mc2/sql-create-context"
    
    LORA_CONFIG = LoraConfig(
        r=16,  # rank
        lora_alpha=32,  # scaling factor
        target_modules=["q_proj", "k_proj", "v_proj"], 
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    MODEL_CACHE_PATH = "/home/common/data/Big_Data/GenAI/llm_models"
    
    lets_finetune(device = device, model = BASE_MODEL, MODEL_PATH = MODEL_PATH, model_cache_path = MODEL_CACHE_PATH, lora_config = LORA_CONFIG, data_path=data_path)
    
    base_model = TextToSQLGenerator(
        base_model_id = model,
        model_cache_path = MODEL_CACHE_PATH,
        use_adapter=False,
        lora_checkpoint="",
    )
    
    USING_CHECKPOINT=100
    LORA_CHECKPOINT = f"./lora_adapters/{BASE_MODEL}/checkpoint-{USING_CHECKPOINT}/"
    if os.path.exists(LORA_CHECKPOINT):
        sample_data = json.loads(samples)
        run_inference(sample_data, model=base_model)
        finetuned_model = TextToSQLGenerator(
            use_adapter=True,
            lora_checkpoint=LORA_CHECKPOINT,
            loaded_base_model=base_model,
            base_model_id = BASE_MODEL,
            model_cache_path = MODEL_CACHE_PATH
        )
        run_inference(sample_data, model=finetuned_model, finetuned=True)
        return finetuned_model

In [11]:
BASE_MODELS = {
    "0": "NousResearch/Nous-Hermes-Llama-2-7b",  # https://huggingface.co/NousResearch/Nous-Hermes-llama-2-7b
    "1": "NousResearch/Llama-2-7b-chat-hf",  # https://huggingface.co/NousResearch/Llama-2-7b-chat-hf
    "2": "NousResearch/Llama-2-13b-hf",  # https://huggingface.co/NousResearch/Llama-2-13b-hf
    "3": "NousResearch/CodeLlama-7b-hf",  # https://huggingface.co/NousResearch/CodeLlama-7b-hf
    "4": "Phind/Phind-CodeLlama-34B-v2",  # https://huggingface.co/Phind/Phind-CodeLlama-34B-v2
    "5": "openlm-research/open_llama_3b_v2",  # https://huggingface.co/openlm-research/open_llama_3b_v2
    "6": "openlm-research/open_llama_13b",  # https://huggingface.co/openlm-research/open_llama_13b
    "7": "HuggingFaceH4/zephyr-7b-beta", # https://huggingface.co/HuggingFaceH4/zephyr-7b-beta
}
# data_path = "b-mc2/sql-create-context"
# data = load_dataset(data_path)

# def prepare_data(data, val_set_size=100) -> Dataset:
#     # """Prepare training and validation datasets."""
#     # train_data = train_val_split["train"]
#     # val_data = train_val_split["test"].shuffle().map(self.tokenize_data)
#     seed = 0
#     temp_a = data['train'].train_test_split(test_size=0.1, seed=seed)
#     temp_b = temp_a['test'].train_test_split(test_size=0.5, seed=seed)

#     return DatasetDict({
#         'train': temp_a['train'].shuffle().map(tokenize_data)],
#         'test': temp_b['test'],
#         'validation': temp_b['train']
#     })

    

# prepared_ds = prepare_data(data)

#for model_name in test_models:
m = generate_model(BASE_MODELS["3"])

Model: NousResearch/CodeLlama-7b-hf
local model path is: /home/common/data/Big_Data/GenAI/llm_models/NousResearch--CodeLlama-7b-hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/78477 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'loss': 2.3162, 'learning_rate': 2e-05, 'epoch': 0.0}
{'eval_loss': 2.4766347408294678, 'eval_runtime': 4.8428, 'eval_samples_per_second': 20.649, 'eval_steps_per_second': 2.684, 'epoch': 0.0}
{'loss': 2.2985, 'learning_rate': 1.5000000000000002e-05, 'epoch': 0.01}
{'eval_loss': 2.2410101890563965, 'eval_runtime': 4.8391, 'eval_samples_per_second': 20.665, 'eval_steps_per_second': 2.686, 'epoch': 0.01}
{'loss': 1.9465, 'learning_rate': 1e-05, 'epoch': 0.01}
{'eval_loss': 1.9794702529907227, 'eval_runtime': 4.8439, 'eval_samples_per_second': 20.644, 'eval_steps_per_second': 2.684, 'epoch': 0.01}
{'loss': 1.7401, 'learning_rate': 5e-06, 'epoch': 0.02}
{'eval_loss': 1.8097870349884033, 'eval_runtime': 4.8516, 'eval_samples_per_second': 20.612, 'eval_steps_per_second': 2.68, 'epoch': 0.02}
{'loss': 1.5892, 'learning_rate': 0.0, 'epoch': 0.02}
{'eval_loss': 1.7447198629379272, 'eval_runtime': 4.834, 'eval_samples_per_second': 20.687, 'eval_steps_per_second': 2.689, 'epoch': 0.02}
{'train_r

...

1
a
b
c
2


1
a
b
c
2


1
a
b
c
2


1
a
b
c
2


1
a
b
c
2


In [12]:
from typing import List
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("juierror/text-to-sql-with-table-schema")
model = AutoModelForSeq2SeqLM.from_pretrained("juierror/text-to-sql-with-table-schema")

def prepare_input(question: str, table: List[str]):
    table_prefix = "table:"
    question_prefix = "question:"
    join_table = ",".join(table)
    inputs = f"{question_prefix} {question} {table_prefix} {join_table}"
    input_ids = tokenizer(inputs, max_length=700, return_tensors="pt").input_ids
    return input_ids

def inference(question: str, table: List[str], model) -> str:
    input_data = prepare_input(question=question, table=table)
    input_data = input_data.to(model.device)
    outputs = model.generate(inputs=input_data, num_beams=10, top_k=10, max_length=700)
    result = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)
    return result

#print(inference(question="get people name with age equal 25", table=["id", "name", "age"], model=model))

In [13]:
q = "What is the capacity of the stadium where the team 'Mountain Eagles' plays?"
t = ['team_name', 'stadium_name', 'capacity']
print(inference(question=q, table=t, model=model))

q = "How many goals did player John Smith score last season?"
t = ['player_name', 'goals_scored', 'season']
print(inference(question=q, table=t, model=model))

q = "What are the operating hours for the Central Library on weekends?"
t = ['library_name', 'day_of_week', 'open_time', 'close_time']
print(inference(question=q, table=t, model=model))

/home/ueda36fa477dc7ba9b01449685e6433d/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


SELECT capacity FROM table WHERE team_name = mountains eagles
SELECT goals_scored FROM table WHERE player_name = john smith AND season = last
SELECT day_of_week FROM table WHERE library_name = central library


In [14]:
# sample_data = json.loads(samples)
# INFERENCE_DEVICE = torch.device("xpu")
# run_inference(sample_data, m, finetuned=True)